<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 2.0 BERT Overview
이 노트북에서는 BERT 아키텍처와 상황에 맞는 단어 임베딩(contextualized word embeddings)에 대해 자세히 살펴보겠습니다. 사전 학습된 언어 모델의 단어들이 서로 어떻게 관련되어 있는지를 시각화할 수 있는 기회를 갖게 됩니다.

**[2.1 Introduction to BERT](#2.1-Introduction-to-BERT)<br>**
**[2.2 BERT Language Model with NeMo](#2.2-BERT-Language-Model-with-NeMo)<br>**
**[2.3 The BERT WordPiece Tokenizers](#2.3-The-BERT-WordPiece-Tokenizers)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.1 Characters, Words, Subwords](#2.3.1-Characters,-Words,-Subwords)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.2 WordPiece Algorithm](#2.3.2-WordPiece-Algorithm)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.3 Tokenizer with NeMo](#2.3.3-Tokenizer-with-NeMo)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.4 Exercise: Tokenizer Index](#2.3.4-Exercise:-Tokenizer-Index)<br>
**[2.4 Contextualized Word Embedding](#2.4-Contextualized-Word-Embedding)<br>**
**[2.5 The Attention Mechanism](#2.5-The-Attention-Mechanism)<br>**


사전 훈련된 언어 모델에 대비하여 BERT 아키텍처를 이해하는 데 중점을 두고 있습니다. 사전 훈련은 다음 실습 노트북에서 다룹니다.  Wikipedia에서 학습된 BERT Base와 같이 비교적 작은 언어 변형 모델임에도 불구하고 트레이닝 과정은 시간이 많이 소요됩니다. 아마도 오늘 수업 시간을 크게 초과하게 됩니다. 따라서 트레이닝 프로세스를 초기화할 뿐 끝까지 진행할 수 없습니다.

사전 훈련된 언어 모델을 사용하는 NLP 미세 조정 작업은 본 코스의 2부에서 다룹니다.

---
# 2.1 Introduction to BERT
["Attention is All You Need!"(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762) 에 소개된 트랜스포머 모델은 처음에는 신경 기계 번역(Neural Machine Translation, NMT) 작업을 위해 사용되었으며, 어텐션 메커니즘에 의존하는 인코더-디코더 아키텍처입니다. 

**BERT**는  **B**idirectional **E**ncoder **R**epresentations from **T**ransformers 를 의미하는 모델로, 트랜스포머 모델의 **인코더 (encoder)** 부분을 기반으로 합니다. BERT는 각 단어 또는 토큰을 학습된, 상황별 표현으로 매핑하여 입력 텍스트를 인코딩합니다.

<center><figure>
    <img src="images/From_Transformer_To_Bert_architecture.png">
    <figcaption>그림 1. 이미지 출처: <a href="https://arxiv.org/abs/1706.03762">Attention is All You Need</a></figcaption>
</figure></center>

특히, BERT 모델은 특별한 토큰 (Special Token) 인 [SEP]로 구분된 두 문장을 입력으로 사용하며, 두 가지 손실 함수를 통해 사전 학습됩니다.
 - 마스킹된 언어 모델 예측 (Masked-language model prediction)
 - 다음 문장 예측 (Next sentence prediction)

Raw 텍스트를 숫자 표현으로 변환하기 위해 BERT 모델은 WordPiece라는 서브워드 토큰화 알고리즘을 사용합니다. 

BERT는 종종 언어 모델 인코더로 사용됩니다. 사전 훈련된 체크포인트는 작업별 추가 레이어들로 확장됩니다 (그림 2 참조). 그런 다음 토큰 분류(이름 있는 엔터티 인식), 텍스트 분류, 질문 답변 등과 같은 다운스트림 작업에서 미세 조정됩니다.

<center><figure>
    <img src="images/BERT.PNG">
    <figcaption>그림 2. 이미지 출처: <a href="https://arxiv.org/pdf/1810.04805.pdf">BERT: Pre-training of Deep Bidirectional Transformers for
        Language Understanding</a>.</figcaption>
</figure></center>

---
# 2.2 BERT Language Model with NeMo

BERT는 널리 사용되는 뉴럴 네트워크이며, 수많은 구현물과 공개적으로 사전 훈련된 체크포인트들을 제공합니다. 

이 예시에서는 [NVIDIA NeMo \(Neural Modules\) Toolkit](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html#neural-modules)을  통해 BERT 사전 훈련된 모델을 사용할 예정입니다. Nemo는 [PyTorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning)을 기반으로 하는 딥 러닝 프레임워크입니다. 파트 2 실습에서는 NeMo에 대해 자세히 설명하겠지만, 지금은 바로 한 번 활용해겠습니다. 먼저 필요한 디펜던시들을 가져오고 사용 가능한 BERT의 변형 모델들을 나열합니다.

In [ ]:
# Import nemo nlp collection 
from nemo.collections import nlp as nemo_nlp

# Import BERT
from nemo.collections.nlp.models import BERTLMModel

In [ ]:
# Check the list of pre-trained BERT language models
BERTLMModel.list_available_models()

NeMo에서는 다음 두 가지 사전 훈련된 BERT 언어 모델을 사용할 수 있습니다. 

-`bertbaseuncased` 모델은 12개의 트랜스포머 블록과 함께 총 1억 1천만 개의 매개 변수를 가집니다.

-`bertlargeuncased` 모델은 24개의 트랜스포머 블록에 총 3억 4천만 개의 매개변수를 가지고 있습니다.

시간과 단순함(simplicity)을 위해 더 작은 변형인 BERT Base를 다운로드합니다. 다운로드는 1~2분 정도 걸릴 예정입니다.

In [ ]:
# Download the pretrained BERT-based model
pretrained_model_name="bertbaseuncased"
model = BERTLMModel.from_pretrained(pretrained_model_name)

다음 셀을 실행하여 모델의 아키텍처를 검사합니다. 나열된 출력 값을 그림 1과 비교하십시오. 

* 모든 층 (Layer)을 식별할 수 있습니까? 
* BERT Base에는 몇 개의 트랜스포머 레이어가 있습니까? 
* 히든 레이어 크기가 얼마나 됩니까? Key, Value 및 Query 매트릭스를 식별할 수 있습니까? 
* 다음 문장 예측과 마스킹된 언어 모델을 담당하는 두 가지 손실(loss) 구성요소를 찾을 수 있습니까?

In [ ]:
# Check the model architecture
model

모델의 크기를 검사합니다.

In [ ]:
# number of weights
print(" Number of weights : ",model.num_weights)

# 2.3 The BERT WordPiece Tokenizers

토큰화는 텍스트 Raw 데이터를 뉴럴 언어 모델에 필요한 이산 숫자 표현으로 변환하는 중요한 데이터 전처리 단계입니다. 문자, 단어 또는 서브워드에 대한 규칙에 따라 텍스트를 토큰으로 분할하는 몇 가지 토큰화 알고리즘이 있습니다. 어휘의 크기는 알고리즘에 의해 결정되며 텍스트 말뭉치(corpus)에서 발견된 토큰의 빈도(frequency)에 따라 달라집니다.

## 2.3.1 Characters, Words, Subwords
토큰화는 단어, 구 또는 더 큰 텍스트 구역을 개별 문자, 단어 또는 서브워드로 분할합니다.  예를 들어, "토큰화"라는 단어는 여러 가지 방법으로 분할될 수 있습니다.

* 문자: 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'
* 단어: 'tokenization'
* 서브워드: 'token', '##ization'

이 아이디어는 텍스트 말뭉치(corpus)에서 토큰의 어휘(vocabulary)를 만들어 토큰 간의 언어 관계를 특징짓기 위해 언어 모델에서 학습될 수 있습니다. 이 작업이 문자, 단어 또는 서브워드로 수행되는지 여부가 문제의 복잡성에 영향을 미칩니다.

문자에 의한 토큰화는 처리할 수 있는 토큰 수가 매우 제한적이라는 장점이 있지만, 이러한 소수의 토큰들은 그 자체로는 큰 의미가 없으며 텍스트를 나타내려면 긴 시퀀스의 토큰이 필요합니다. 단어에 의한 토큰화는 매우 큰 어휘 크기를 초래하고 매우 유사한 단어에 대해 별도의 토큰을 필요로 하며, 이는 서로의 관계를 결정하기 위한 더 많은 학습을 필요로 합니다.

서브워드에 의한 토큰화는 이 두 개의 균형을 맞추기 위한 솔루션입니다. 예를 들어 "token"이라는 단어는 "tokenization", "tokens" 및 "tokenize"의 하위 단어입니다.  단어를 분할함으로써 모델은 동일한 어근 단어에서 유사한 의미를 더 쉽게 학습할 수 있습니다.  이해하는 데 필요한 전체 어휘의 크기가 단어 토큰화에 필요한 크기보다 작습니다.

## 2.3.2 WordPiece Algorithm
WordPiece 알고리즘은 [Schuster 와 Nakajima 논문](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf) 에 소개되었습니다.  먼저 학습 데이터(corpus)와 원하는 서브워드 어휘 크기를 선택합니다. 알고리즘은 텍스트 본문에 대한 최적의 서브워드를 반복적으로 결정하고 값이 할당된 어휘를 생성합니다. 반복되는 단계는 다음과 같습니다.

1. 단어를 문자 토큰의 시퀀스로 나눕니다.
2. 이전 단계의 토큰을 사용하여 학습 데이터를 기반으로 언어 모델을 작성합니다.
3. 언어 모델에서 가능성이 높은 두 개의 토큰을 결합하여 새 유닛 토큰을 생성하고 새 토큰을 어휘에 추가합니다.
4. 2단계부터 원하는 어휘의 토큰 제한에 도달하거나 가능성이 원하는 임계값 아래로 떨어질 때까지 반복합니다.

## 2.3.3 Tokenizer with NeMo

In [ ]:
# Check available tokenizers
nemo_nlp.modules.get_tokenizer_list()

In [ ]:
# Get the bert-base-uncased tokenizer 
tokenizer_uncased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-uncased")

In [ ]:
# Check the vocabulary size
print(" The vocabulary size: ", tokenizer_uncased.vocab_size)

In [ ]:
SAMPLES_TEXT_1 = "Hello, my name is John. I live in Santa Clara."

In [ ]:
output_uncased=tokenizer_uncased.text_to_tokens(SAMPLES_TEXT_1)
print("Input sentence: ", SAMPLES_TEXT_1)
print("Tokenized sentence: ", output_uncased)

이제`bert-base-cased` 토크나이저를 사용하여 "Hello, my name is John. I live in Santa Clara" 문장을 인코딩 해봅니다.

In [ ]:
# Get the bert-base-cased tokenizer 
tokenizer_cased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-cased")

In [ ]:
# Encode the text 
output_cased=tokenizer_cased.text_to_tokens(SAMPLES_TEXT_1)
print("Input sentence: ", SAMPLES_TEXT_1)
print("Tokenized sentence: ", output_cased)

BERT 모델은 텍스트 입력이 아니라 숫자 인덱스 표현을 허용합니다.

`tokenizer.text_to_ids()` 함수를 사용하여 단어의 어휘 인덱스를 확인할 수 있습니다. 

이번에는 `bert-base-cased` 토크나이저를 사용하여 실험해 보십시오.

In [ ]:
# Index of the tokens Hello and hello using bert-base-cased tokenizer
print("Index of Hello: ", tokenizer_cased.text_to_ids("Hello"))
print("Index of hello: ",tokenizer_cased.text_to_ids("hello"))

In [ ]:
# Example of bert-base-cased tokenizer in a sentence
print("Input sentence: ", SAMPLES_TEXT_1)
print("Tokenized sentence: ", output_cased)
print("Tokenized sentence: ", tokenizer_cased.text_to_ids(SAMPLES_TEXT_1))

## 2.3.4 Exercise: Tokenizer Index
노트북 앞 부분에서 이미 `tokenizer_uncased`를 설정해 두었습니다. 아래 셀에서 "hello"에 대한 인덱스와 `bert-base-uncased`에 대한 "Hello"를 출력하십시오. 어려우면 [솔루션](solutions/ex2.3.4.ipynb)을 확인하십시오.

In [ ]:
# Index of the tokens Hello and hello using bert-base-uncased tokenizer
print("Index of Hello: ") #FIXME
print("Index of hello: ") #FIXME

---
# 2.4 Contextualized Word Embedding

BERT 모델은 일단 학습이 되면 언어 말뭉치(corpus)에서 함께 사용을 기반으로 토큰화된 단어 간의 관계를 제공합니다.  이러한 관계는 신경 망의 숨겨진 상태인 _문맥화된 단어 임베딩(contextualized word embeddings)_ 내에서 정의됩니다.  이러한 관계는 텍스트 분류 (Text Classification), 명명된 개체 인식 (Named Entity Recognition), 질문 답변 (Q&A) 등과 같은 NLP 작업을 해결하는 데 사용할 수 있습니다.  
임베딩을 시각화하려면 "mouse"라는 단어를 두 가지 이상의 뜻으로 사용하는 문장으로 시작하여 사전 훈련된 모델이 무엇을 만들 수 있는지 살펴보겠습니다.

In [ ]:
import torch

# Set up the sentence we want to look at
TEXT = "Last night, my wireless mouse was eaten by an animal such as mouse or rat. I need to order a new optical computer mouse."
input_sentence=torch.tensor([tokenizer_uncased.tokenizer(TEXT).input_ids]).cuda()
attention_mask=torch.tensor([tokenizer_uncased.tokenizer(TEXT).attention_mask]).cuda()

In [ ]:
# Show the tokenization for the sentence
print("Input sentence: ", TEXT)
output_uncased=tokenizer_uncased.ids_to_tokens(tokenizer_uncased.tokenizer(TEXT).input_ids)
print("Tokenized sentence: ", output_uncased)

# "mouse" tokens positions in the TEXT input
mouse_computer_1=6
mouse_animal=14
mouse_computer_2=26

In [ ]:
# Get the embeddings for the pretrained model
hidden_states = model.bert_model(input_ids=input_sentence, token_type_ids=None, attention_mask=attention_mask)

In [ ]:
from numpy import dot
from numpy.linalg import norm

def similarity_cosine(x,y):
    return dot(x,y)/(norm(x)*norm(y))

먼저 차원성을 2D로 줄이면 BERT 모델에서 얻은 텍스트 토큰 임베딩을 시각화할 수 있습니다. 
[t-SNE](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)는 주변 거리를 보존하여 2D 또는 3D 벡터 시각화에 널리 사용되는 차원 축소 기법입니다.

이 그림에서는 t-SNE를 사용하여 차원성을 2D 차원으로 축소시킨 후 텍스트 토큰 BERT 임베딩이 적용된 플롯의 예를 보여 줍니다.
<img src="images/Embeddings.PNG" width=800>

직접 사용해 보세요!  

다음 코드 블록은 [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) 을 사용하여 t-SNE 알고리즘을 통해 BERT 텍스트 토큰 임베딩의 차원성을 768에서 2로 줄이고 2D 벡터를 플롯합니다. t-SNE는 확률적 과정이기 때문에 저차원 임베딩은 각 실행마다 다릅니다. 그러나 토큰의 주변 거리는 거의 동일하게 유지되어야 합니다.

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

X=hidden_states.cpu().detach().numpy()
X_embedded = TSNE(n_components=2,metric='euclidean',  init='random', perplexity=7).fit_transform(X[0])
Tokens=tokenizer_uncased.ids_to_tokens(tokenizer_uncased.tokenizer(TEXT).input_ids)

# Annotate the different mouse tokens
Tokens[mouse_computer_1]="mouse_computer_1"
Tokens[mouse_animal]="mouse_animal"
Tokens[mouse_computer_2]="mouse_computer_2"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(15,10))
plt.plot(X_embedded[:,0],X_embedded[:,1], '.', color='black')
plt.plot([X_embedded[6,0],X_embedded[14,0]],[X_embedded[6,1],X_embedded[14,1]],color='red')
plt.plot([X_embedded[6,0],X_embedded[26,0]],[X_embedded[6,1],X_embedded[26,1]],color='green')

for i, txt in enumerate(Tokens):
    plt.annotate(txt, (X_embedded[i,0], X_embedded[i,1]), color='blue')

---
# 2.5 The Attention Mechanism

트랜스포머 아키텍처에 대한 이전 노트북에서 설명한 바와 같이, 어텐션 메커니즘을 통해 모델에서는 시퀀스의 특정 부분에 *참여하거나* 초점을 맞출 수 있으며, 더 관련이 있는 부분에 더 높은 값을 할당할 수 있습니다. 인간의 어텐션 메커니즘에서 영감을 얻었음에도 불구하고, 트랜스포머에서 구현되는 방법은 비교적 간단합니다. Key 매트릭스와 Query 매트릭스를 곱하는 것으로 구성됩니다. 또한 레이어당 여러 헤드(BERT Base의 경우 레이어당 12 헤드)가 포함되며, 이러한 헤드 중 어느 것도 사람이 해석할 수 있는 결과를 제공하도록 조정되지 않습니다. 따라서 어텐션 메커니즘을 해석하는 것은 어려울 수 있습니다.  뉴럴 네트워크는 모든 레이어와 헤드에 걸쳐 매우 많은 텍스트 상호작용과 패턴을 학습합니다. 그럼에도 불구하고 [BertViz](https://github.com/jessevig/bertviz)라는 도구를 사용하여 시각화해보도록 하겠습니다. 

이번 섹션에서는 "유럽 경제 영역(european economic area)"(참고: https://www.gov.uk/eu-eea) 과 같이 어텐션 메커니즘을 과장하기 위해 복잡하고 여러 토큰의 용어를 사용하는 문장을 의도적으로 구성할 예정입니다.

In [ ]:
from visualization import head, KVQ

In [ ]:
sentence_a = "The European Union (EU) is an economic and political union of 27 countries."
sentence_b = "The European Economic Area (EEA) The EEA includes EU countries and also Iceland, Liechtenstein and Norway."

In [ ]:
# Run the head attention visualization tool - this may take a few minutes
head.berthead(sentence_a,sentence_b)

In [ ]:
# Run the KVQ matrix visualization tool - this may take a few minutes
KVQ.bertKVQ(sentence_a, sentence_b)

---
<h2 style="color:green;">축하합니다!</h2>

여러분은 다음을 배웠습니다.
* BERT는 트랜스포머 인코더를 기반으로 하는 언어 모델입니다.
* WordPiece 토크나이저는 서브워드를 수학적 표현으로 변환합니다.
* BERT 모델에서 단어 간의 관계는 임베딩을 2개의 차원으로 줄이고 플롯하여 시각화할 수 있습니다.
* 어텐션 메커니즘은 더 높은 값을 할당하여 시퀀스의 특정 부분에 초점을 맞춥니다.

다음으로 여러분은 토큰화 방법 및 BERT 모델 학습에 대한 기본을 배울 예정입니다.  [3.0 Pretraining Language Models](030_PretrainingLM.ipynb)로 이동해 주세요.

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>